In [1]:
library(FlashRLearn)
fm.set.conf("/FlashX/EC2/run_matrix.txt")

Loading required package: pcg
Loading required package: FlashR
Loading required package: RSpectra
Loading required package: Rcpp

Attaching package: 'FlashR'

The following objects are masked from 'package:base':

    cbind, pmax, pmin, rbind



In [ ]:
fm.set.test.naest.naest.na(FALSE)

# Load metadata of the MRN dataset from S3

In [2]:
df <- read.csv(url("http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/ndmg.csv"))
files <- read.csv(url("http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/files.txt"), header=FALSE)

# Load data from S3

In [3]:
vs <- list()
mats <- list()
num <- 0
labels <- rep.int(0, length(df$Sex))
for (idx in 1:length(df$URSI)) {
    id <- as.character(df$URSI[idx])
    file <- paste("MRN114_", id, "_1_DTI_aligned.dat.gz", sep="")
    path <- paste("http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/",
                  file, sep="")
    if (file %in% files$V1) {
        print(path)
        flush.console()
        conn <- gzcon(url(path))
        v <- fm.load.dense.matrix.bin(conn, TRUE, 505472240, 1, FALSE, "F", id)
        if (!is.null(v)) {
            num <- num + 1
            vs <- c(vs, v)
            labels[num] <- df$Sex[idx]
        }
        close(conn)
    }
    if (length(vs) >= 16) {
        mat <- fm.cbind.list(vs)
        vs <- list()
        mat_name <- paste("MRN-", length(mats), ".mat", sep="")
        print(paste("store as ", mat_name, sep=""))
        flush.console()
        mat <- fm.conv.store(mat, in.mem=FALSE, name=mat_name)
        mats <- c(mats, mat)
        gc()
    }
}
if (length(vs) > 0) {
    mat <- fm.cbind.list(vs)
    vs <- list()
    mat_name <- paste("MRN-", length(mats), ".mat", sep="")
    print(paste("store as ", mat_name, sep=""))
    flush.console()
    mat <- fm.conv.store(mat, in.mem = FALSE, name=mat_name)
    mats <- c(mats, mat)
    gc()
}
data <- fm.cbind.list(mats)


[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87102217_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87102806_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87103074_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87105476_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87107085_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87108094_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87111487_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87111924_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87114047_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-w

[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87164886_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87165017_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87165441_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87166115_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87168759_1_DTI_aligned.dat.gz"
[1] "store as MRN-4.mat"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87174803_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87176019_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87176708_1_DTI_aligned.dat.gz"
[1] "http://neurodata-public-lol.s3-website-us-east-1.amazonaws.com/MRN114_M87178630_1_DTI_aligned.dat.gz"
[1] "http://

In [4]:
data <- fm.conv.store(data, in.mem=FALSE, name="MRN.mat")
save(labels, file = "/FlashX/MRN_labels.Rdata")

ERROR: Error in save(labels, "/FlashX/MRN_labels.Rdata"): object '/FlashX/MRN_labels.Rdata' not found


In [6]:
print(paste("#samples:", ncol(data)))
print(paste("#features:", nrow(data)))
print(paste("size in bytes:", length(data) * 8))

[1] "#samples: 112"
[1] "#features: 505472240"
[1] "size in bytes: 452903127040"


In [7]:
embed.classifier <- function(data, labels, proj=c("LOL", "LAL", "QOQ", "PCA"), red.p=1)
{
	if (proj == "LOL")
		proj <- LOL(data, fm.conv.R2FM(as.integer(labels)), red.p, type="svd")
	else if (proj == "LAL")
		proj <- LOL(data, fm.conv.R2FM(as.integer(labels)), red.p, type="rand_sparse")
	else if (proj == "QOQ")
		proj <- QOQ(data, fm.conv.R2FM(as.integer(labels)), red.p)
	else if (proj == "PCA") {
		mu <- rowMeans(data)
		center.mat <- sweep(data, 1, mu, "-")
		res <- fm.svd(t(center.mat), red.p, red.p)
		proj <- res$v
	}
}

train.classifier <- function(data, labels, proj, method="lda")
{
	proj.res <- t(data) %*% proj
	if (method == "lda")
		res <- lda(as.matrix(fm.conv.FM2R(proj.res)), as.factor(labels))
	else if (method == "qda")
		res <- qda(as.matrix(fm.conv.FM2R(proj.res)), as.factor(labels))
	else
		res <- NULL
	list(proj=proj, res=res)
}

predict.classifier <- function(object, newdata)
{
	proj.res <- t(newdata) %*% object$proj
	predict(object=object$res, newdata=as.matrix(fm.conv.FM2R(proj.res)))
}

rand.split.test <- function(data, labels, count, train.percent, red.ps)
{
	train.size <- as.integer(ncol(data) * train.percent)
	for (run in 1:count) {
		idxs <- 1:ncol(data)
		train.idxs <- sort(sample(idxs, train.size))
		test.idxs <- which(is.na(pmatch(idxs, train.idxs)))
		train <- data[,train.idxs]
		test <- data[,test.idxs]
		train.labels <- labels[train.idxs]
		truth <- labels[test.idxs]+1
		print("truth:")
		print(truth)
        flush.console()

		proj <- embed.classifier(train, train.labels, proj="LOL", max(red.ps))
		for (red.p in red.ps) {
			res <- train.classifier(train, train.labels, proj[, 1:red.p], method="lda")
			pred <- predict.classifier(object=res, newdata=test)
			print("LOL+LDA predict:")
			print(pred$class)
			# measure the accuracy
			out <- paste("LOL-", red.p, "dim: ",
						 sum((as.integer(pred$class) - truth) != 0)/length(pred$class), sep="")
			print(out)
            flush.console()

			res <- train.classifier(train, train.labels, proj[, 2:min(red.p + 1, ncol(proj))], method="lda")
			pred <- predict.classifier(object=res, newdata=test)
			print("RR-LDA predict:")
			print(pred$class)
			# measure the accuracy
			out <- paste("RR-LDA-", red.p, "dim: ",
						 sum((as.integer(pred$class) - truth) != 0)/length(pred$class), sep="")
			print(out)
            flush.console()
			res <- NULL
			gc()
		}
		proj <- NULL
		gc()

		proj <- embed.classifier(train, train.labels, proj="PCA", max(red.ps))
		for (red.p in red.ps) {
            red.p <- min(red.p, ncol(proj))
			res <- train.classifier(train, train.labels, proj[, 1:red.p], method="lda")
			pred <- predict.classifier(object=res, newdata=test)
			print("PCA+LDA predict:")
			print(pred$class)
			# measure the accuracy
			out <- paste("PCA-", red.p, "dim: ",
						 sum((as.integer(pred$class) - truth) != 0)/length(pred$class), sep="")
			print(out)
            flush.console()
			res <- NULL
			gc()
		}
		proj <- NULL
		gc()
	}
}

In [8]:
library(MASS)
rand.split.test(data, labels, 1, 0.9, c(1, seq(10, 100, 10)))

[1] "truth:"
 [1] 2 2 1 1 2 1 1 1 1 2 2 1
[1] "LOL+LDA predict:"
 [1] 1 1 1 1 1 1 1 1 0 1 1 0
Levels: 0 1
[1] "LOL-1dim: 0.416666666666667"
[1] "RR-LDA predict:"
 [1] 1 1 1 1 1 1 1 1 0 1 1 1
Levels: 0 1
[1] "RR-LDA-1dim: 0.5"
[1] "LOL+LDA predict:"
 [1] 1 1 1 0 1 0 1 0 0 1 1 0
Levels: 0 1
[1] "LOL-10dim: 0.166666666666667"
[1] "RR-LDA predict:"
 [1] 1 1 0 0 1 0 0 1 0 1 1 0
Levels: 0 1
[1] "RR-LDA-10dim: 0.0833333333333333"
[1] "LOL+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "LOL-20dim: 0.0833333333333333"
[1] "RR-LDA predict:"
 [1] 1 1 0 0 1 0 1 1 0 1 1 0
Levels: 0 1
[1] "RR-LDA-20dim: 0.166666666666667"
[1] "LOL+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "LOL-30dim: 0.0833333333333333"
[1] "RR-LDA predict:"
 [1] 1 1 1 0 1 0 1 1 0 1 1 0
Levels: 0 1
[1] "RR-LDA-30dim: 0.25"
[1] "LOL+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "LOL-40dim: 0.0833333333333333"
[1] "RR-LDA predict:"
 [1] 1 1 0 0 1 0 1 1 0 1 1 0
Levels: 0 1
[1] "RR-LDA-40dim: 0.

Warning message in lda.default(x, grouping, ...):
"variables are collinear"

[1] "LOL+LDA predict:"
 [1] 1 1 1 0 1 0 1 1 0 1 1 0
Levels: 0 1
[1] "LOL-100dim: 0.25"


Warning message in lda.default(x, grouping, ...):
"variables are collinear"

[1] "RR-LDA predict:"
 [1] 1 1 1 0 1 0 1 1 0 1 1 0
Levels: 0 1
[1] "RR-LDA-100dim: 0.25"
[1] "PCA+LDA predict:"
 [1] 1 1 1 1 1 1 1 1 0 1 1 1
Levels: 0 1
[1] "PCA-1dim: 0.5"
[1] "PCA+LDA predict:"
 [1] 1 1 0 0 1 0 0 1 0 1 1 0
Levels: 0 1
[1] "PCA-10dim: 0.0833333333333333"
[1] "PCA+LDA predict:"
 [1] 1 1 0 0 1 0 1 1 0 1 1 0
Levels: 0 1
[1] "PCA-20dim: 0.166666666666667"
[1] "PCA+LDA predict:"
 [1] 1 1 1 0 1 0 0 1 0 1 1 0
Levels: 0 1
[1] "PCA-30dim: 0.166666666666667"
[1] "PCA+LDA predict:"
 [1] 1 1 1 0 1 0 0 1 0 1 1 0
Levels: 0 1
[1] "PCA-40dim: 0.166666666666667"
[1] "PCA+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "PCA-50dim: 0.0833333333333333"
[1] "PCA+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "PCA-60dim: 0.0833333333333333"
[1] "PCA+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "PCA-70dim: 0.0833333333333333"
[1] "PCA+LDA predict:"
 [1] 1 1 1 0 1 0 0 0 0 1 1 0
Levels: 0 1
[1] "PCA-80dim: 0.0833333333333333"
[1] "PCA+LDA predict:"
 [1] 1 

Warning message in lda.default(x, grouping, ...):
"variables are collinear"

[1] "PCA+LDA predict:"
 [1] 1 1 1 1 1 1 1 1 1 1 1 1
Levels: 0 1
[1] "PCA-100dim: 0.583333333333333"
